predicts the claim_type to a given premise from the trained BERT model

In [33]:
from transformers import TextClassificationPipeline
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import pickle
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [34]:
claim_type_frame = pickle.load( open( "pickles/claim_type_frame_bt_et.p", "rb" ) )
att_frame_list = pickle.load( open( "pickles/att_frame_list.p", "rb" ) )
ent_frame_list = pickle.load( open( "pickles/ent_frame_list.p", "rb" ) )
rel_frame_list = pickle.load( open( "pickles/rel_frame_list.p", "rb" ) )

In [35]:
 #load model

possible_labels = claim_type_frame.Values.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


model.load_state_dict(torch.load('Modelle/finetuned_BERT_epoch_3_frame_3_b8.model', map_location=torch.device('cpu')))


#load tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [36]:
frame_list=[]
#class predictor
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)

#for i in range(1,5):
for i in range(len(ent_frame_list)):
    
    #make a frame containing only premises
    p=pd.merge(att_frame_list[i],ent_frame_list[i],on="Targets")
    
    p=p.loc[p["Type_x"]=="PremiseType"]

    #get the text from all premises
    text_list = p['text'].tolist()

    label_list=[]
    for j in text_list:
        #predict label
        res=pipe(j)

        #make result as dictonary
        hold = [item for sublist in res for item in sublist]
        result = {}
        for k in hold:
            result.update(dict([tuple(k.values())]))

        label_list.append(max(result, key=result.get))

    label_list

    #add label to dataframe
    p["pred_label"]=label_list


    #make frame to get the true values by adding realtions from rel list
    d=att_frame_list[i]
    d=d.loc[d["Type"]=="ClaimType"]


    g=pd.merge(rel_frame_list[i],ent_frame_list[i],on="Targets")
    g=pd.merge(g,ent_frame_list[i],left_on="Supported",right_on="Targets")
    g=pd.merge(g,d,left_on="Supported",right_on="Targets",how="left")


    g=g.drop(["text_x","text_y","Targets_y","Targets","Type","ID_y"],axis=1)
    g=g.rename(columns={"Targets_x":"Targets","Type_x":"Type_Target","Type_y":"Type_Supported"})

    #prediction is onyl over premise types drop all claim support claim relations
    g = g.drop(g[(g["Type_Target"] =="Claim") & (g["Type_Supported"]=="Claim")].index)

    
    #combine predicted and true labels
    t=pd.merge(g,p,left_on="Targets",right_on="Targets")
    t=t.rename(columns={"Values_x":"True_Values"})
    t=t.drop(["ID_x","ID","Type_x","text","Type_y","Values_y"],axis=1)
    
    t["pred_label"].replace("LABEL_0","Value",inplace=True)
    t["pred_label"].replace("LABEL_1","Fact",inplace=True)
    t["pred_label"].replace("LABEL_2","Policy",inplace=True)
    
    frame_list.append(t)

In [37]:
frame_list[0]

,Stance,Targets,Supported,Type_Target,Type_Supported,True_Values,pred_label
0,supports,T5,T3,Premise,Claim,Value,Fact
1,supports,T6,T3,Premise,Claim,Value,Value
2,supports,T10,T11,Premise,Claim,Fact,Value
3,supports,T8,T7,Premise,Claim,Fact,Fact


In [38]:
big_frame = pd.concat(frame_list)
big_frame
big_frame.groupby(["True_Values","pred_label"]).count()

Stance  Targets  Supported  Type_Target  \
True_Values pred_label                                            
Fact        Fact           119      119        119          119   
            Value          318      318        318          318   
Policy      Fact            26       26         26           26   
            Value           20       20         20           20   
Value       Fact            53       53         53           53   
            Value           66       66         66           66   

                        Type_Supported  
True_Values pred_label                  
Fact        Fact                   119  
            Value                  318  
Policy      Fact                    26  
            Value                   20  
Value       Fact                    53  
            Value                   66

In [39]:
big_frame["True_Values"].value_counts()

Fact      437
Value     119
Policy     46
Name: True_Values, dtype: int64

In [40]:
big_frame["pred_label"].value_counts()

Value    485
Fact     222
Name: pred_label, dtype: int64

In [41]:
y_gold= big_frame['True_Values'].tolist()
y_pred=big_frame["pred_label"].tolist()

print("acc",accuracy_score(y_gold,y_pred))

print("f1",f1_score(y_gold,y_pred,average="macro"))

acc 0.26166902404526166
f1 0.144924077219146
